In [16]:
import time
import torch
from torch.utils.data import DataLoader, TensorDataset
from torchtext.data.utils import get_tokenizer
from torchtext.datasets import DATASETS
from torchtext.vocab import build_vocab_from_iterator
import torch.nn as nn
from tqdm import tqdm
import pickle
import random
import numpy as np
from collections import Counter, defaultdict
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
import gensim.downloader
from torch import FloatTensor as FT

# Get the interactive Tools for Matplotlib
%matplotlib notebook
%matplotlib inline

plt.style.use('ggplot')

### Instructions
For this part, fill in the required code and make the notebook work. This wll be very similar to the Skip-Gram model, but a little more difficult. Look for the """ FILL IN """ string to guide you.

In [17]:
# Where do I want to run my job. You can do "cuda" on linux machines
DEVICE = "mps" if torch.backends.mps.is_available() else  "cpu"
# DEVICE = "cuda" if torch.cuda.is_available() else  "cpu"

# The batch size in Adam or SGD
BATCH_SIZE = 512

# Number of epochs
NUM_EPOCHS = 10

# Predict from 2 words the inner word for CBOW
# I.e. I'll have a window like ["a", "b", "c"] of continuous text (each is a word)
# We'll predict each of wc = ["a", "c"] from "b" = wc for Skip-Gram
# For CBOW, we'll use ["a", "c"] to predict "b" = wo
WINDOW = 1

# Negative samples.
K = 4

The text8 Wikipedia corpus. 100M characters.

In [18]:
# Put the data in your Google Drive
# You ca get the data here: https://www.kaggle.com/competitions/titanic/data
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
!du -h text8


du: cannot access 'text8': No such file or directory


In [20]:

f = open('/content/drive/MyDrive/titanic/text8 (1)', 'r')
text = f.read()
# One big string of size 100M
print(len(text))

100000000


In [21]:
punc = '!"#$%&()*+,-./:;<=>?@[\\]^_\'{|}~\t\n'

# Can do regular expressions here too
for c in punc:
    if c in text:
        text.replace(c, ' ')

In [22]:
# A very crude tokenizer you get for free: lower case and also split on spaces
TOKENIZER = get_tokenizer("basic_english")

In [23]:
#text= text[0:int(len(text)/4)]

In [24]:
words = TOKENIZER(text)
f = Counter(words)

In [25]:
len(words)

17005207

In [26]:
# Do a very crude filter on the text which removes all very popular words
text = [word for word in words if f[word] > 5]

In [27]:
text[0:5]

['anarchism', 'originated', 'as', 'a', 'term']

In [28]:
VOCAB = build_vocab_from_iterator([text])

In [29]:
# word -> int hash map
stoi = VOCAB.get_stoi()
# int -> word hash map
itos = VOCAB.get_itos()

In [30]:
stoi['as']

11

In [31]:
itos[11]

'as'

In [32]:
# Total number of words
len(stoi)

63641

In [33]:
f = Counter(text)
# This is the probability that we pick a word in the corpus
z = {word: f[word] / len(text) for word in f}

In [34]:
threshold = 1e-5
# Probability that word is kept while subsampling
# This is explained here and sightly differet from the paper: http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/
p_keep = {word: (np.sqrt(z[word] / 0.001) + 1)*(0.0001 / z[word]) for word in f}

In [35]:
# This is in the integer space
train_dataset = [word for word in text if random.random() < p_keep[word]]

# Rebuild the vocabulary
VOCAB = build_vocab_from_iterator([train_dataset])

In [36]:
len(train_dataset)

7848473

In [37]:
# word -> int mapping
stoi = VOCAB.get_stoi()
# int -> word mapping
itos = VOCAB.get_itos()

In [38]:
# The vocabulary size after we do all the filters
len(VOCAB)

63641

In [39]:
# The probability we draw something for negative sampling
f = Counter(train_dataset)
p = torch.zeros(len(VOCAB))

# Downsample frequent words and upsample less frequent
s = sum([np.power(freq, 0.75) for word, freq in f.items()])

for word in f:
    p[stoi[word]] = np.power(f[word], 0.75) / s

In [ ]:
p

In [ ]:
stoi["as"]

In [ ]:
# Map everything to integers
train_dataset = [stoi[word] for word in text]

In [ ]:
train_dataset

In [33]:
def get_tokenized_dataset(dataset, verbose=False):
    x_list = []

    for i, token in enumerate(dataset):
      m = 1

      # Get the left and right tokens
      start = max(i - m, 0)
      left_tokens = dataset[start:i]

      end = min(i + m + 1, len(dataset))
      right_tokens = dataset[i + 1:end]

      # Check these are the same length, and if so use them to add a row of data. This should be a list like
      # [a, c, b] where b is the center word
      if len(left_tokens) == len(right_tokens):

          w_context = [left_tokens[0], right_tokens[0]]


          wc = dataset[i]
          #print(wc)

          w_context.append(wc)

          print(w_context)

          x_list.extend(
              [w_context]
          )

    return x_list


In [34]:
train_x_list = get_tokenized_dataset(train_dataset, verbose=False)

Streaming output truncated to the last 5000 lines.
[57447, 1880, 6699]
[6699, 157, 1880]
[1880, 33, 157]
[157, 2854, 33]
[33, 159, 2854]
[2854, 114, 159]
[159, 35, 114]
[114, 1299, 35]
[35, 29, 1299]
[1299, 663, 29]
[29, 820, 663]
[663, 48, 820]
[820, 881, 48]
[48, 4, 881]
[881, 527, 4]
[4, 3, 527]
[527, 8, 3]
[3, 17, 8]
[8, 15, 17]
[17, 2, 15]
[15, 817, 2]
[2, 6, 817]
[817, 6699, 6]
[6, 16, 6699]
[6699, 330, 16]
[16, 449, 330]
[330, 1, 449]
[449, 57447, 1]
[1, 4, 57447]
[57447, 534, 4]
[4, 748, 534]
[534, 62, 748]
[748, 35, 62]
[62, 6149, 35]
[35, 4, 6149]
[6149, 3, 4]
[4, 8, 3]
[3, 17, 8]
[8, 17, 17]
[17, 6699, 17]
[17, 1232, 6699]
[6699, 0, 1232]
[1232, 15092, 0]
[0, 4, 15092]
[15092, 3, 4]
[4, 8, 3]
[3, 17, 8]
[8, 9, 17]
[17, 33, 9]
[9, 0, 33]
[33, 244, 0]
[0, 1, 244]
[244, 9, 1]
[1, 3, 9]
[9, 4, 3]
[3, 3, 4]
[4, 8, 3]
[3, 22, 8]
[8, 22, 22]
[22, 33, 22]
[22, 244, 33]
[33, 15, 244]
[244, 17, 15]
[15, 35, 17]
[17, 2280, 35]
[35, 5, 2280]
[2280, 4626, 5]
[5, 919, 4626]
[4626, 11, 919

In [ ]:

type(train_dataset)

In [35]:
train_x_list

[[5233, 11, 3083],
 [3083, 5, 11],
 [11, 183, 5],
 [5, 1, 183],
 [183, 3133, 1],
 [1, 46, 3133],
 [3133, 56, 46],
 [46, 163, 56],
 [56, 125, 163],
 [163, 741, 125],
 [125, 523, 741],
 [741, 10619, 523],
 [523, 115, 10619],
 [10619, 0, 115],
 [115, 27497, 0],
 [0, 1, 27497],
 [27497, 0, 1],
 [1, 113, 0],
 [0, 854, 113],
 [113, 2, 854],
 [854, 0, 2],
 [2, 15177, 0],
 [0, 59203, 15177],
 [15177, 1, 59203],
 [59203, 0, 1],
 [1, 124, 0],
 [0, 854, 124],
 [124, 3583, 854],
 [854, 0, 3583],
 [3583, 183, 0],
 [0, 10, 183],
 [183, 180, 10],
 [10, 56, 180],
 [180, 4, 56],
 [56, 5, 4],
 [4, 10751, 5],
 [5, 218, 10751],
 [10751, 6, 218],
 [218, 1324, 6],
 [6, 101, 1324],
 [1324, 457, 101],
 [101, 19, 457],
 [457, 56, 19],
 [19, 2733, 56],
 [56, 327, 2733],
 [2733, 6, 327],
 [327, 3673, 6],
 [6, 0, 3673],
 [3673, 708, 0],
 [0, 1, 708],
 [708, 398, 1],
 [1, 27, 398],
 [398, 40, 27],
 [27, 37, 40],
 [40, 57, 37],
 [37, 554, 57],
 [57, 109, 554],
 [554, 11, 109],
 [109, 5, 11],
 [11, 1425, 5],
 [5, 27

In [36]:
pickle.dump(train_x_list, open('train_x_list.pkl', 'wb'))

HERE

In [5]:
train_x_list = pickle.load(open('/content/drive/MyDrive/train_x_list.pkl', 'rb'))

In [6]:
train_x_list

[[5233, 11, 3083],
 [3083, 5, 11],
 [11, 183, 5],
 [5, 1, 183],
 [183, 3133, 1],
 [1, 46, 3133],
 [3133, 56, 46],
 [46, 163, 56],
 [56, 125, 163],
 [163, 741, 125],
 [125, 523, 741],
 [741, 10619, 523],
 [523, 115, 10619],
 [10619, 0, 115],
 [115, 27497, 0],
 [0, 1, 27497],
 [27497, 0, 1],
 [1, 113, 0],
 [0, 854, 113],
 [113, 2, 854],
 [854, 0, 2],
 [2, 15177, 0],
 [0, 59203, 15177],
 [15177, 1, 59203],
 [59203, 0, 1],
 [1, 124, 0],
 [0, 854, 124],
 [124, 3583, 854],
 [854, 0, 3583],
 [3583, 183, 0],
 [0, 10, 183],
 [183, 180, 10],
 [10, 56, 180],
 [180, 4, 56],
 [56, 5, 4],
 [4, 10751, 5],
 [5, 218, 10751],
 [10751, 6, 218],
 [218, 1324, 6],
 [6, 101, 1324],
 [1324, 457, 101],
 [101, 19, 457],
 [457, 56, 19],
 [19, 2733, 56],
 [56, 327, 2733],
 [2733, 6, 327],
 [327, 3673, 6],
 [6, 0, 3673],
 [3673, 708, 0],
 [0, 1, 708],
 [708, 398, 1],
 [1, 27, 398],
 [398, 40, 27],
 [27, 37, 40],
 [40, 57, 37],
 [37, 554, 57],
 [57, 109, 554],
 [554, 11, 109],
 [109, 5, 11],
 [11, 1425, 5],
 [5, 27

In [7]:
# These are (wc, wo) pairs. All are y = +1 by design
train_x_list

[[5233, 11, 3083],
 [3083, 5, 11],
 [11, 183, 5],
 [5, 1, 183],
 [183, 3133, 1],
 [1, 46, 3133],
 [3133, 56, 46],
 [46, 163, 56],
 [56, 125, 163],
 [163, 741, 125],
 [125, 523, 741],
 [741, 10619, 523],
 [523, 115, 10619],
 [10619, 0, 115],
 [115, 27497, 0],
 [0, 1, 27497],
 [27497, 0, 1],
 [1, 113, 0],
 [0, 854, 113],
 [113, 2, 854],
 [854, 0, 2],
 [2, 15177, 0],
 [0, 59203, 15177],
 [15177, 1, 59203],
 [59203, 0, 1],
 [1, 124, 0],
 [0, 854, 124],
 [124, 3583, 854],
 [854, 0, 3583],
 [3583, 183, 0],
 [0, 10, 183],
 [183, 180, 10],
 [10, 56, 180],
 [180, 4, 56],
 [56, 5, 4],
 [4, 10751, 5],
 [5, 218, 10751],
 [10751, 6, 218],
 [218, 1324, 6],
 [6, 101, 1324],
 [1324, 457, 101],
 [101, 19, 457],
 [457, 56, 19],
 [19, 2733, 56],
 [56, 327, 2733],
 [2733, 6, 327],
 [327, 3673, 6],
 [6, 0, 3673],
 [3673, 708, 0],
 [0, 1, 708],
 [708, 398, 1],
 [1, 27, 398],
 [398, 40, 27],
 [27, 37, 40],
 [40, 57, 37],
 [37, 554, 57],
 [57, 109, 554],
 [554, 11, 109],
 [109, 5, 11],
 [11, 1425, 5],
 [5, 27

In [8]:
# The number of things of BATCH_SIZE = 512
assert(len(train_x_list) // BATCH_SIZE == 32579)

In [9]:
print(type(train_x_list))


<class 'list'>


### Set up the dataloader.

In [10]:
train_dl = DataLoader(
    TensorDataset(
        torch.tensor(train_x_list).to(DEVICE),
    ),
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [11]:
for xb in train_dl:
    assert(xb[0].shape == (BATCH_SIZE, 3))
    break

### Words we'll use to asses the quality of the model ...

In [44]:
valid_ids = torch.tensor([
    stoi['money'],
    stoi['lion'],
    stoi['africa'],
    stoi['musician'],
    stoi['dance'],
])

### Get the model.

In [45]:
class CBOWNegativeSampling(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(CBOWNegativeSampling, self).__init__()
        self.A = nn.Embedding(vocab_size, embed_dim) # Context vectors - center word
        self.B = nn.Embedding(vocab_size, embed_dim) # Output vectors - words around the center word
        self.init_weights()

    def init_weights(self):
        # Is this the best way? Not sure
        initrange = 0.5
        self.A.weight.data.uniform_(-initrange, initrange)
        self.B.weight.data.uniform_(-initrange, initrange)

    def forward(self, x):
        # N is the batch size
        # x is (N, 3)

        # Context words are 2m things, m = 1 so w_context is (N, 2) while wc is (N, 1)
        w_context, wc = x[:, 0:2], x[:, 2]



        # Each of these is (N, 2, D) since each context has 2 word
        # We want this to be (N, D) and this is what we get

        # (N, 2, D)
        a = self.A(w_context)

        # (N, D)
        a_avg = torch.mean(a, dim=1)


        # Each of these is (N, D) since each target has 1 word
        b = self.A(wc)

        # The product between each context and target vector. Look at the Skip-Gram code.
        # The logits is now (N, 1) since we sum across the final dimension.
        logits = (a * b).sum(axis=-1)

        return logits

In [46]:
class CBOWNegativeSampling(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(CBOWNegativeSampling, self).__init__()
        self.A = nn.Embedding(vocab_size, embed_dim)  # Context vectors - center word.
        self.B = nn.Embedding(vocab_size, embed_dim)  # Output vectors - words around the center word
        self.init_weights()

    def init_weights(self):
        # Is this the best way? Not sure
        initrange = 0.5
        self.A.weight.data.uniform_(-initrange, initrange)
        self.B.weight.data.uniform_(-initrange, initrange)

    def forward(self, x):
        # N is the batch size
        # x is (N, 3)
        # Context words are 2m things, m = 1 so w_context is (N, 2) while wc is (N, 1)
        w_context, wc = x[:, :-1], x[:, -1]
        # Each of these is (N, 2, D) since each context has 2 word
        # We want this to be (N, D) and this is what we get
        # (N, 2, D)
        a = self.A(w_context)
        # (N, D)
        a_avg = a.mean(axis=1)
        # Each of these is (N, D) since each target has 1 word
        b = self.B(wc)
        # The product between each context and target vector. Look at the Skip-Gram code.
        # The logits is now (N, 1) since we sum across the final dimension.
        logits = (a_avg * b).sum(axis=-1)
        return logits


In [47]:
@torch.no_grad()
def validate_embeddings(
    model,
    valid_ids,
    itos
):
    """ Validation logic """

    # We will use context embeddings to get the most similar words
    # Other strategies include: using target embeddings, mean embeddings after avaraging context/target
    embedding_weights = model.A.weight

    normalized_embeddings = embedding_weights.cpu() / np.sqrt(
        np.sum(embedding_weights.cpu().numpy()**2, axis=1, keepdims=True)
    )

    # Get the embeddings corresponding to valid_term_ids
    valid_embeddings = normalized_embeddings[valid_ids, :]

    # Compute the similarity between valid_term_ids (S) and all the embeddings (V)
    # We do S x d (d x V) => S x D and sort by negative similarity
    top_k = 10 # Top k items will be displayed
    similarity = np.dot(valid_embeddings.cpu().numpy(), normalized_embeddings.cpu().numpy().T)

    # Invert similarity matrix to negative
    # Ignore the first one because that would be the same word as the probe word
    similarity_top_k = np.argsort(-similarity, axis=1)[:, 1: top_k+1]

    # Print the output.
    for i, word_id in enumerate(valid_ids):
        # j >= 1 here since we don't want to include the word itself.
        similar_word_str = ', '.join([itos[j] for j in similarity_top_k[i, :] if j >= 1])
        print(f"{itos[word_id]}: {similar_word_str}")

    print('\n')

### Set up the model

In [40]:
LR = 10.0
NUM_EPOCHS = 10
EMBED_DIM = 300

In [41]:
model = CBOWNegativeSampling(len(VOCAB), EMBED_DIM).to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

# The learning rate is lowered every epoch by 1/10
# Is this a good idea?
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.1)

In [42]:
model

CBOWNegativeSampling(
  (A): Embedding(63641, 300)
  (B): Embedding(63641, 300)
)

In [48]:
validate_embeddings(model, valid_ids, itos)

money: interrelated, reconciled, submarine, moulding, despise, ewing, hourly, undocked, ojibwa, dicke
lion: significantly, paris, scripting, fishmonger, thai, egoism, doodle, indoor, swell, samson
africa: archive, actinopterygii, ahriman, anicetus, denaturation, jasper, modena, sacagawea, redirected, coastlines
musician: purveyor, applying, permaculture, impose, butch, geelong, mails, embellishment, knoxville, expedited
dance: lees, anarcha, preface, intentional, cvc, falcon, balding, biting, nontechnical, mind




### Train the model

In [53]:
ratios = []

def train(dataloader, model, optimizer, epoch):
    model.train()
    total_acc, total_count, total_loss, total_batches = 0, 0, 0.0, 0.0
    log_interval = 500

    for idx, x_batch in tqdm(enumerate(dataloader)):
        x_batch = x_batch[0]
        batch_size = x_batch.shape[0]

        # Zero the gradient so they don't accumulate
        optimizer.zero_grad()

        logits = model(x_batch)

        # Get the positive samples loss. Notice we use weights here
        positive_loss = torch.nn.BCEWithLogitsLoss()(input=logits, target=torch.ones(batch_size).to(DEVICE).float())

        # For each batch, get some negative samples
        # We need a total of len(y_batch) * K samples across a batch
        # We then reshape this batch
        # These are effectively the output words
        negative_samples = torch.multinomial(p, batch_size * K, replacement=True)

        # Context words are 2m things, m = 1 so w_context is (N, 2) while wc is (N, 1)
        w_context, wc = x_batch[:, :-1], x_batch[:, -1]

        """
        if w_context looks like below (batch_size = 3)
        [
        (a, b),
        (c, d),
        (e, f)
        ] and K = 2 we'd like to get:
        [
        (a, b),
        (a, b),
        (c, d),
        (c, d),
        (e, f),
        (e, f)
        ]
        This will be batch_size * K rows.
        """

        # This should be (N * K, 2)
        w_context = torch.concat([w.repeat(K, 1) for w in torch.tensor(w_context).split(1)])

        # Remove the last dimension 1
        wc = negative_samples.unsqueeze(-1)

        # Get the negative samples. This should be (N * K, 3)
        # Concatenate the w_context and wc along the column. Make sure everything is on CUDA / MPS or CPU
        x_batch_negative = torch.concat([w_context, wc.to(DEVICE)], axis=1)

        """
        Note the way we formulated the targets: they are all 0 since these are negative samples.
        We do the BCEWithLogitsLoss by hand basically here.
        Notice we sum across the negative samples, per positive word.
        This is literally the equation in the lecture notes.
        """

        # (N, K, D) -> (N, D) -> (N)
        # Look at the Skip-Gram notebook
        negative_loss = model(x_batch_negative).neg().sigmoid().log().reshape(batch_size, K).sum(1).mean().neg().to(DEVICE)
        loss = (positive_loss + negative_loss).mean()

        # Get the gradients via back propagation.
        loss.backward()

        # Do an optimization step.
        optimizer.step()

        total_loss += loss.item()
        total_batches += 1

        if idx % log_interval == 0:
            print("| epoch {:3d} | {:5d}/{:5d} batches | loss {:8.3f} ".format(epoch, idx, len(dataloader), total_loss / total_batches))
            validate_embeddings(model, valid_ids, itos)
            total_loss, total_batches = 0.0, 0.0


### Some results from the run look like below:

Somewhere inside of 2 iterations you should get sensible associattions.
Paste here a screenshot of the closest vectors.

In [ ]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()

    train(train_dl, model, optimizer, epoch)
    # We have a learning rate scheduler here

    # Basically, given the state of the optimizer, this lowers the learning rate in a smart way
    scheduler.step()

0it [00:00, ?it/s]<ipython-input-53-448348fcc4aa>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w_context = torch.concat([w.repeat(K, 1) for w in torch.tensor(w_context).split(1)])
1it [00:01,  1.67s/it]

| epoch   1 |     0/32580 batches | loss    2.106 
money: days, being, city, early, using, group, science, down, c, east
lion: significantly, paris, scripting, egoism, indoor, doodle, swell, thai, coherency, adding
africa: name, although, nine, by, early, six, so, two, four, after
musician: support, analogy, september, purveyor, v, under, presidio, maximilian, geelong, applying
dance: view, mind, preface, subject, having, algol, formed, orange, biting, spent




501it [01:31,  4.95it/s]

| epoch   1 |   500/32580 batches | loss    1.971 
money: days, being, early, city, using, group, east, science, down, court
lion: significantly, paris, scripting, egoism, indoor, doodle, swell, coherency, adding, thai
africa: name, nine, early, although, by, eight, t, with, after, so
musician: support, september, v, name, language, analogy, under, purveyor, one, while
dance: view, mind, having, subject, period, preface, name, spent, power, formed




1001it [02:56,  5.18it/s]

...

25501it [1:14:53,  5.18it/s]

| epoch   4 | 25500/32580 batches | loss    1.176 
money: then, form, court, chinese, days, it, slow, lack, importance, something
lion: unable, paris, johnson, policy, patrick, husband, hands, rest, ussr, experiment
africa: external, irish, community, rock, michael, peter, women, thomas, available, dr
musician: culture, singer, actress, poet, composer, writer, action, mathematician, politician, physicist
dance: area, my, musical, folk, pop, jazz, end, blues, musician, literary




26001it [1:16:19,  4.39it/s]

| epoch   4 | 26000/32580 batches | loss    1.174 
money: then, form, court, chinese, days, it, slow, lack, importance, something
lion: unable, paris, johnson, policy, patrick, husband, hands, rest, ussr, experiment
africa: external, irish, community, rock, michael, peter, women, thomas, available, dr
musician: culture, singer, actress, poet, composer, writer, action, mathematician, politician, physicist
dance: area, my, musical, folk, pop, jazz, end, blues, musician, literary




26254it [1:17:03,  6.00it/s]